In [4]:
import pandas as pd
import json
import os

In [5]:
# From 2021-06-07 two zeros were cut off the price
def clean_data_sbmx(sbmx, cutoff):
    sbmx_cleaned = []
    duplicate = set()
    for i in sbmx:
        if i['date'] in duplicate: continue

        if (pd.to_datetime(i['date']) < cutoff):
            for price in i.keys():
                if 'price' in price:
                    if i[price] == '-': 
                        i[price] = i['price_at_closure']
                    if float(i[price]) < 100: continue
                    i[price] = str(round(float(i[price]) / 100, 3))
        i['volume_of_trade'] = i['volume_of_trade'].replace('-', '0')
        sbmx_cleaned.append(i)     
        duplicate.add(i['date'])
        
    return sbmx_cleaned

def clean_data_other(data):
    data_cleaned = []
    duplicate = set()
    for i in data:
        if i['date'] in duplicate: continue

        for price in i.keys():
            if 'price' in price:
                if i[price] == '-': 
                    i[price] = i['price_at_closure']
        
        i['volume_of_trade'] = i['volume_of_trade'].replace('-', '0')
        data_cleaned.append(i)     
        duplicate.add(i['date'])

    return data_cleaned
    

In [6]:
data_folder = '../data/'
with open(data_folder + 'SBMX.json', 'r+') as file:
    sbmx = json.load(file)
    sbmx_cleaned = clean_data_sbmx(sbmx, pd.to_datetime('2021-06-06'))

    # Overwrite the file with cleaned data
    file.seek(0)
    json.dump(sbmx_cleaned, file, indent=4)
    file.truncate()


for filename in os.listdir(data_folder):
    if filename.endswith('.json') and filename != 'SBMX.json':
        file_path = os.path.join(data_folder, filename)
        
        # Open and clean the JSON file
        with open(file_path, 'r+') as file:
            data = json.load(file)
            cleaned_data = clean_data_other(data)
            
            # Overwrite the file with cleaned data
            file.seek(0)
            json.dump(cleaned_data, file, indent=4)
            file.truncate()
    